In [ ]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.6 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
import os

# --- Kannada Character Classes ---
CLASSES = {
    # 🪷 Swaras (15)
    "swaras": [
        ("a", "ಅ", "0C85"), ("aa", "ಆ", "0C86"), ("i", "ಇ", "0C87"), ("ii", "ಈ", "0C88"),
        ("u", "ಉ", "0C89"), ("uu", "ಊ", "0C8A"), ("ru", "ಋ", "0C8B"), ("e", "ಎ", "0C8E"),
        ("ee", "ಏ", "0C8F"), ("ai", "ಐ", "0C90"), ("o", "ಒ", "0C92"), ("oo", "ಓ", "0C93"),
        ("au", "ಔ", "0C94"), ("am", "ಅಂ", "0C85+0C82"), ("aha", "ಅಃ", "0C85+0C83")
    ],

    # 🧱 Vyanjanas (34)
    "vyanjanas": [
        ("ka", "ಕ", "0C95"), ("kha", "ಖ", "0C96"), ("ga", "ಗ", "0C97"), ("gha", "ಘ", "0C98"), ("nga", "ಙ", "0C99"),
        ("cha", "ಚ", "0C9A"), ("chha", "ಛ", "0C9B"), ("ja", "ಜ", "0C9C"), ("jha", "ಝ", "0C9D"), ("nya", "ಞ", "0C9E"),
        ("ta_rt", "ಟ", "0C9F"), ("tha_rt", "ಠ", "0CA0"), ("da_rt", "ಡ", "0CA1"), ("dha_rt", "ಢ", "0CA2"), ("na_rt", "ಣ", "0CA3"),
        ("ta_dn", "ತ", "0CA4"), ("tha_dn", "ಥ", "0CA5"), ("da_dn", "ದ", "0CA6"), ("dha_dn", "ಧ", "0CA7"), ("na_dn", "ನ", "0CA8"),
        ("pa", "ಪ", "0CAA"), ("pha", "ಫ", "0CAB"), ("ba", "ಬ", "0CAC"), ("bha", "ಭ", "0CAD"), ("ma", "ಮ", "0CAE"),
        ("ya", "ಯ", "0CAF"), ("ra", "ರ", "0CB0"), ("la", "ಲ", "0CB2"), ("va", "ವ", "0CB5"),
        ("lla", "ಳ", "0CB3"), ("sha", "ಶ", "0CB6"), ("ssa", "ಷ", "0CB7"), ("sa", "ಸ", "0CB8"), ("ha", "ಹ", "0CB9")
    ],

    # 🎯 Matras (15 modifiers) — visual symbols used to modify consonants
    "matras": [
        ("aa_matra", "ಾ", ""), ("i_matra", "ಿ", ""), ("ii_matra", "ೀ", ""), ("u_matra", "ು", ""), ("uu_matra", "ೂ", ""),
        ("ru_matra", "ೃ", ""), ("e_matra", "ೆ", ""), ("ee_matra", "ೇ", ""), ("ai_matra", "ೈ", ""), ("o_matra", "ೊ", ""),
        ("oo_matra", "ೋ", ""), ("au_matra", "ೌ", ""), ("am_matra", "ಂ", ""), ("aha_matra", "ಃ", ""), ("virama", "್", "")
    ],

    # 🪷 Gunitaksharas (32 samples covering different swaras applied to consonants)
    "gunitaksharas": [
        ("kaa", "ಕಾ", ""), ("ki", "ಕಿ", ""), ("kii", "ಕೀ", ""), ("ku", "ಕು", ""), ("kuu", "ಕೂ", ""),
        ("kai", "ಕೈ", ""), ("ke", "ಕೆ", ""), ("ko", "ಕೊ", ""), ("kho", "ಖೊ", ""), ("gaai", "ಗೈ", ""),
        ("gi", "ಗಿ", ""), ("gu", "ಗು", ""), ("chi", "ಚಿ", ""), ("chu", "ಚು", ""), ("jaa", "ಜಾ", ""),
        ("ji", "ಜಿ", ""), ("ta_rt_i", "ಟಿ", ""), ("ta_dn_i", "ತಿ", ""), ("da_dn_uu", "ದು", ""), ("na_dn_ai", "ನೈ", ""),
        ("pa_i", "ಪಿ", ""), ("pa_oo", "ಪೋ", ""), ("ba_i", "ಬಿ", ""), ("ba_ee", "ಬೇ", ""), ("ma_oo", "ಮೋ", ""),
        ("ya_e", "ಯೆ", ""), ("ra_ai", "ರೈ", ""), ("la_o", "ಲೊ", ""), ("va_e", "ವೆ", ""), ("sa_i", "ಸಿ", ""),
        ("sha_i", "ಶಿ", ""), ("ha_ee", "ಹೇ", "")
    ],

    # 🔠 Ottaksharas (65 common conjunct consonants)
    "vottaksharas": [
        # Common consonant conjuncts
        ("kka", "ಕ್ಕ", ""), ("kta", "ಕ್ತ", ""), ("kna", "ಕ್ನ", ""), ("kra", "ಕ್ರ", ""), ("kma", "ಕ್ಮ", ""),
        ("gda", "ಗ್ದ", ""), ("gla", "ಗ್ಲ", ""), ("gra", "ಗ್ರ", ""), ("gga", "ಗ್ಗ", ""), ("gna", "ಗ್ನ", ""),
        ("cha_cha", "ಚ್ಚ", ""), ("chra", "ಛ್ರ", ""), ("jja", "ಜ್ಜ", ""), ("jra", "ಜ್ರ", ""), ("jna", "ಜ್ಞ", ""),
        ("tta", "ಟ್ಟ", ""), ("tna", "ಟ್ನ", ""), ("tra", "ತ್ರ", ""), ("tma", "ತ್ಮ", ""), ("tda", "ತ್ದ", ""),
        ("dda", "ದ್ದ", ""), ("dga", "ದ್ಗ", ""), ("dya", "ದ್ಯ", ""), ("dra", "ದ್ರ", ""), ("dha_na", "ಧಾನ", ""),
        ("ppa", "ಪ್ಪ", ""), ("pra", "ಪ್ರ", ""), ("pla", "ಪ್ಲ", ""), ("bba", "ಬ್ಬ", ""), ("bra", "ಬ್ರ", ""),
        ("bha_ra", "ಭ್ರ", ""), ("mra", "ಮ್ರ", ""), ("mma", "ಮ್ಮ", ""), ("mna", "ಮ್ನ", ""), ("ya_ya", "ಯ್ಯ", ""),
        ("yra", "ಯ್ರ", ""), ("ra_ra", "ರ್ರ", ""), ("rma", "ರ್ಮ", ""), ("rna", "ರ್ನ", ""), ("la_la", "ಲ್ಲ", ""),
        ("lra", "ಲ್ರ", ""), ("vra", "ವ್ರ", ""), ("ssa_ra", "ಷ್ರ", ""), ("sha_ra", "ಶ್ರ", ""), ("sna", "ಸ್ನ", ""),
        ("spa", "ಸ್ಪ", ""), ("ska", "ಸ್ಕ", ""), ("sma", "ಸ್ಮ", ""), ("hra", "ಹ್ರ", ""), ("hna", "ಹ್ನ", ""),
        ("hva", "ಹ್ವ", ""), ("hla", "ಹ್ಲ", ""), ("lla", "ಳ್ಳ", ""), ("nta", "ನ್ತ", ""), ("nda", "ನ್ದ", ""),
        ("nra", "ನ್ರ", ""), ("ksha", "ಕ್ಷ", ""), ("shcha", "ಶ್ಚ", ""), ("shna", "ಶ್ನ", ""), ("shma", "ಶ್ಮ", ""),
        ("dva", "ದ್ವ", ""), ("tva", "ತ್ವ", ""), ("bha_ra_alt", "ಭ್ರ", ""), ("sra", "ಸ್ರ", ""), ("kra_alt", "ಕ್ರ", "")
    ]
}


# --- Output directory ---
OUTPUT_DIR = "kannada_sheets_clean_50"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_sheet(char_name, char_symbol, unicode_val, category, index):
    sheet_id = f"{category.upper()}_{index:03d}"
    file_path = os.path.join(OUTPUT_DIR, f"{sheet_id}_{char_name}.pdf")

    c = canvas.Canvas(file_path, pagesize=A4)
    width, height = A4
    margin = 15 * mm

    # --- Header ---
    c.setFont("Helvetica-Bold", 16)
    c.drawCentredString(width / 2, height - 25 * mm, "Kannada Handwritten Dataset Collection")

    # Show Unicode in header for later mapping
    c.setFont("Helvetica", 12)
    c.drawCentredString(width / 2, height - 35 * mm,
                        f"Character: {char_name} | Unicode: {unicode_val} | Category: {category}")

    # --- Grid settings (same as before) ---
    rows, cols = 5, 10         # 50 boxes per sheet
    box_width = 16 * mm
    box_height = 30 * mm
    spacing_x = 2 * mm
    spacing_y = 4 * mm

    total_grid_width = cols * box_width + (cols - 1) * spacing_x
    total_grid_height = rows * box_height + (rows - 1) * spacing_y

    # Center grid horizontally and vertically under header
    start_x = (width - total_grid_width) / 2
    header_space = 50 * mm
    start_y = height - header_space - 10  # slight padding

    # --- Draw empty boxes (no character inside) ---
    for i in range(rows):
        for j in range(cols):
            x = start_x + j * (box_width + spacing_x)
            y = start_y - i * (box_height + spacing_y) - box_height
            c.rect(x, y, box_width, box_height)

    c.showPage()
    c.save()
    print(f"✅ Created: {file_path}")


# --- Generate sheets ---
counter = 1
for category, items in CLASSES.items():
    for i, (name, symbol, uni) in enumerate(items, start=1):
        create_sheet(name, symbol, uni, category, counter)
        counter += 1

print(f"\n🎉 Total {counter-1} clean sheets generated in '{OUTPUT_DIR}' folder.")


✅ Created: kannada_sheets_clean_50/SWARAS_001_a.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_002_aa.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_003_i.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_004_ii.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_005_u.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_006_uu.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_007_ru.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_008_e.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_009_ee.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_010_ai.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_011_o.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_012_oo.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_013_au.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_014_am.pdf
✅ Created: kannada_sheets_clean_50/SWARAS_015_aha.pdf
✅ Created: kannada_sheets_clean_50/VYANJANAS_016_ka.pdf
✅ Created: kannada_sheets_clean_50/VYANJANAS_017_kha.pdf
✅ Created: kannada_sheets_clean_50/VYANJANAS_018_ga.pdf
✅ Created: kannada_sheets_clean_50/VYANJ

In [ ]:
import shutil
from google.colab import files

# --- Path to the folder containing PDFs ---
folder_to_zip = "kannada_sheets_clean_50"
zip_filename = "kannada_sheets_clean_50.zip"

# --- Create a zip file ---
shutil.make_archive(zip_filename.replace(".zip",""), 'zip', folder_to_zip)

# --- Download the zip ---
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import zipfile
from math import ceil

# Directory where all PDFs are stored
INPUT_DIR = "kannada_sheets_clean_50"

# Output directory for zips
OUTPUT_ZIP_DIR = "kannada_sheets_zips"
os.makedirs(OUTPUT_ZIP_DIR, exist_ok=True)

# Assign people
people = ["shri", "lavanya", "arati", "akash"]

# List all PDF files sorted alphabetically
pdf_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith(".pdf")])

# Divide PDFs evenly among 4 people
num_files = len(pdf_files)
split_size = ceil(num_files / len(people))

for idx, person in enumerate(people):
    start = idx * split_size
    end = start + split_size
    person_files = pdf_files[start:end]

    zip_path = os.path.join(OUTPUT_ZIP_DIR, f"{person}_kannada_sheets.zip")
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for pdf in person_files:
            pdf_path = os.path.join(INPUT_DIR, pdf)
            zipf.write(pdf_path, arcname=pdf)
    print(f"✅ Created ZIP for {person}: {zip_path}")

print(f"\n🎉 All done! 4 zip files created in '{OUTPUT_ZIP_DIR}' folder.")


✅ Created ZIP for shri: kannada_sheets_zips/shri_kannada_sheets.zip
✅ Created ZIP for lavanya: kannada_sheets_zips/lavanya_kannada_sheets.zip
✅ Created ZIP for arati: kannada_sheets_zips/arati_kannada_sheets.zip
✅ Created ZIP for akash: kannada_sheets_zips/akash_kannada_sheets.zip

🎉 All done! 4 zip files created in 'kannada_sheets_zips' folder.


In [ ]:
import shutil
from google.colab import files
import os
from math import ceil

# --- Input folder with PDFs ---
INPUT_DIR = "kannada_sheets_clean_50"

# --- People and ZIP folder names ---
people = ["shri", "lavanya", "arati", "akash"]

# --- List PDFs sorted alphabetically ---
pdf_files = sorted([f for f in os.listdir(INPUT_DIR) if f.endswith(".pdf")])
num_files = len(pdf_files)
split_size = ceil(num_files / len(people))

# --- Create separate ZIPs for each person ---
for idx, person in enumerate(people):
    start = idx * split_size
    end = start + split_size
    person_files = pdf_files[start:end]

    # --- Temporary folder for this person's PDFs ---
    temp_folder = f"temp_{person}"
    os.makedirs(temp_folder, exist_ok=True)

    # --- Copy PDFs to temp folder ---
    for pdf in person_files:
        shutil.copy(os.path.join(INPUT_DIR, pdf), temp_folder)

    # --- Make ZIP ---
    zip_filename = f"{person}_kannada_sheets.zip"
    shutil.make_archive(zip_filename.replace(".zip",""), 'zip', temp_folder)

    # --- Download ZIP ---
    files.download(zip_filename)

    # --- Clean up temp folder ---
    shutil.rmtree(temp_folder)
    print(f"✅ Created and downloaded ZIP for {person}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Created and downloaded ZIP for shri


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

✅ Created and downloaded ZIP for arati


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Created and downloaded ZIP for akash


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from math import ceil, sqrt

# Total number of boxes
TOTAL_BOXES = 161

# Split into two pages: 80 + 81
page_boxes = [80, 81]

def draw_empty_page(c, n_boxes, page_title):
    width, height = A4
    margin = 10*mm
    spacing = 4*mm

    # estimate columns and rows
    cols = ceil(sqrt(n_boxes * width / height))
    rows = ceil(n_boxes / cols)

    # compute box size
    box_width = (width - 2*margin - (cols-1)*spacing) / cols
    box_height = (height - 2*margin - 20*mm - (rows-1)*spacing) / rows
    box_size = min(box_width, box_height)

    # starting positions
    total_width = cols * box_size + (cols-1)*spacing
    x_start = (width - total_width)/2
    y_start = height - margin - 15*mm

    # page title
    c.setFont("Helvetica-Bold", 16)
    c.drawCentredString(width/2, height - margin, page_title)

    idx = 0
    for r in range(rows):
        y = y_start - r*(box_size + spacing)
        for col in range(cols):
            if idx >= n_boxes:
                break
            x = x_start + col*(box_size + spacing)
            c.rect(x, y - box_size, box_size, box_size)  # draw empty box
            idx += 1

OUTPUT_FILE = "kannada_161_empty_boxes.pdf"
c = canvas.Canvas(OUTPUT_FILE, pagesize=A4)

draw_empty_page(c, page_boxes[0], "Kannada Dataset Collection - Page 1")
c.showPage()
draw_empty_page(c, page_boxes[1], "Kannada Dataset Collection - Page 2")
c.showPage()

c.save()
print(f"✅ PDF generated: {OUTPUT_FILE}")


✅ PDF generated: kannada_161_empty_boxes.pdf


In [ ]:
from google.colab import files
files.download("kannada_161_empty_boxes.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from math import ceil
import json
import os

# ---------------------------
# Define characters for two sheets
# ---------------------------
SHEET1_CHARS = [
    # Row-wise, left-to-right, top-to-bottom for sheet 1
    ("ಅ","swaras"),("ಆ","swaras"),("ಇ","swaras"),("ಈ","swaras"),("ಉ","swaras"),("ಊ","swaras"),("ಋ","swaras"),("ಎ","swaras"),
    ("ಏ","swaras"),("ಐ","swaras"),("ಒ","swaras"),("ಓ","swaras"),("ಔ","swaras"),("ಅಂ","swaras"),("ಅಃ","swaras"),("ಕ","vyanjanas"),
    ("ಖ","vyanjanas"),("ಗ","vyanjanas"),("ಘ","vyanjanas"),("ಙ","vyanjanas"),("ಚ","vyanjanas"),("ಛ","vyanjanas"),("ಜ","vyanjanas"),("ಝ","vyanjanas"),
    ("ಞ","vyanjanas"),("ಟ","vyanjanas"),("ಠ","vyanjanas"),("ಡ","vyanjanas"),("ಢ","vyanjanas"),("ಣ","vyanjanas"),("ತ","vyanjanas"),("ಥ","vyanjanas"),
    ("ದ","vyanjanas"),("ಧ","vyanjanas"),("ನ","vyanjanas"),("ಪ","vyanjanas"),("ಫ","vyanjanas"),("ಬ","vyanjanas"),("ಭ","vyanjanas"),("ಮ","vyanjanas"),
    ("ಯ","vyanjanas"),("ರ","vyanjanas"),("ಲ","vyanjanas"),("ವ","vyanjanas"),("ಳ","vyanjanas"),("ಶ","vyanjanas"),("ಷ","vyanjanas"),("ಸ","vyanjanas"),
    ("ಹ","vyanjanas"),("ಾ","matras"),("ಿ","matras"),("ೀ","matras"),("ು","matras"),("ೂ","matras"),("ೃ","matras"),("ೆ","matras"),("ೇ","matras"),
    ("ೈ","matras"),("ೊ","matras"),("ೋ","matras"),("ೌ","matras"),("ಂ","matras"),("ಃ","matras"),("್","matras"),("ಕಾ","gunitaksharas"),
]

SHEET2_CHARS = [
    # Remaining 16 gunitaksharas + 65 vottaksharas
    ("ಕಿ","gunitaksharas"),("ಕೀ","gunitaksharas"),("ಕು","gunitaksharas"),("ಕೂ","gunitaksharas"),("ಕೈ","gunitaksharas"),
    ("ಕೆ","gunitaksharas"),("ಕೊ","gunitaksharas"),("ಖೊ","gunitaksharas"),("ಗೈ","gunitaksharas"),("ಗಿ","gunitaksharas"),
    ("ಗು","gunitaksharas"),("ಚಿ","gunitaksharas"),("ಚು","gunitaksharas"),("ಜಾ","gunitaksharas"),("ಜಿ","gunitaksharas"),("ಟಿ","gunitaksharas"),
    # 65 Ottaksharas (row-wise)
    ("ತಿ","gunitaksharas"),("ದು","gunitaksharas"),("ನೈ","gunitaksharas"),("ಪಿ","gunitaksharas"),("ಪೋ","gunitaksharas"),("ಬಿ","gunitaksharas"),("ಬೇ","gunitaksharas"),("ಮೋ","gunitaksharas"),
    ("ಯೆ","gunitaksharas"),("ರೈ","gunitaksharas"),("ಲೊ","gunitaksharas"),("ವೆ","gunitaksharas"),("ಸಿ","gunitaksharas"),("ಶಿ","gunitaksharas"),("ಹೇ","gunitaksharas"),("ಕ್ಕ","vottaksharas"),
    ("ಕ್ತ","vottaksharas"),("ಕ್ನ","vottaksharas"),("ಕ್ರ","vottaksharas"),("ಕ್ಮ","vottaksharas"),("ಗ್ದ","vottaksharas"),("ಗ್ಲ","vottaksharas"),("ಗ್ರ","vottaksharas"),("ಗ್ಗ","vottaksharas"),
    ("ಗ್ನ","vottaksharas"),("ಚ್ಚ","vottaksharas"),("ಛ್ರ","vottaksharas"),("ಜ್ಜ","vottaksharas"),("ಜ್ರ","vottaksharas"),("ಜ್ಞ","vottaksharas"),("ಟ್ಟ","vottaksharas"),("ಟ್ನ","vottaksharas"),
    ("ತ್ರ","vottaksharas"),("ತ್ಮ","vottaksharas"),("ತ್ದ","vottaksharas"),("ದ್ದ","vottaksharas"),("ದ್ಗ","vottaksharas"),("ದ್ಯ","vottaksharas"),("ದ್ರ","vottaksharas"),("ಧಾನ","vottaksharas"),
    ("ಪ್ಪ","vottaksharas"),("ಪ್ರ","vottaksharas"),("ಪ್ಲ","vottaksharas"),("ಬ್ಬ","vottaksharas"),("ಬ್ರ","vottaksharas"),("ಭ್ರ","vottaksharas"),("ಮ್ರ","vottaksharas"),("ಮ್ಮ","vottaksharas"),
    ("ಮ್ನ","vottaksharas"),("ಯ್ಯ","vottaksharas"),("ಯ್ರ","vottaksharas"),("ರ್ರ","vottaksharas"),("ರ್ಮ","vottaksharas"),("ರ್ನ","vottaksharas"),("ಲ್ಲ","vottaksharas"),("ಲ್ರ","vottaksharas"),
    ("ವ್ರ","vottaksharas"),("ಷ್ರ","vottaksharas"),("ಶ್ರ","vottaksharas"),("ಸ್ನ","vottaksharas"),("ಸ್ಪ","vottaksharas"),("ಸ್ಕ","vottaksharas"),("ಸ್ಮ","vottaksharas"),("ಹ್ರ","vottaksharas"),
    ("ಹ್ನ","vottaksharas"),("ಹ್ವ","vottaksharas"),("ಹ್ಲ","vottaksharas"),("ಳ್ಳ","vottaksharas"),("ನ್ತ","vottaksharas"),("ನ್ದ","vottaksharas"),("ನ್ರ","vottaksharas"),("ಕ್ಷ","vottaksharas"),
    ("ಶ್ಚ","vottaksharas"),("ಶ್ನ","vottaksharas"),("ಶ್ಮ","vottaksharas"),("ದ್ವ","vottaksharas"),("ತ್ವ","vottaksharas"),("ಭ್ರ_ALT","vottaksharas"),("ಸ್ರ","vottaksharas"),("ಕ್ರ_ALT","vottaksharas")
]

# ---------------------------
# PDF page size and box layout
# ---------------------------
PAGE_WIDTH, PAGE_HEIGHT = A4
MARGIN = 10*mm
SPACING = 4*mm
ROWS = 10
COLS = 8

def generate_mapping(chars):
    mapping = []
    n = len(chars)
    # calculate rows dynamically
    rows = ceil(n / COLS)
    # box width and height
    box_width = (PAGE_WIDTH - 2*MARGIN - (COLS-1)*SPACING) / COLS
    box_height = (PAGE_HEIGHT - 2*MARGIN - (ROWS-1)*SPACING - 15*mm)/ROWS
    x_start = MARGIN
    y_start = PAGE_HEIGHT - MARGIN

    idx = 0
    for r in range(rows):
        y = y_start - r*(box_height + SPACING)
        for c_idx in range(COLS):
            if idx >= n:
                break
            x = x_start + c_idx*(box_width + SPACING)
            mapping.append({
                "name": chars[idx][0],
                "category": chars[idx][1],
                "coords": [x, y - box_height, box_width, box_height]
            })
            idx += 1
    return mapping

# Generate mapping for both sheets
mapping_json = {
    "page1": generate_mapping(SHEET1_CHARS),
    "page2": generate_mapping(SHEET2_CHARS)
}

# Save mapping JSON
os.makedirs("output_mapping", exist_ok=True)
with open("output_mapping/kannada_box_mapping.json", "w", encoding="utf-8") as f:
    json.dump(mapping_json, f, ensure_ascii=False, indent=4)

print("✅ Mapping JSON generated: output_mapping/kannada_box_mapping.json")


✅ Mapping JSON generated: output_mapping/kannada_box_mapping.json


In [ ]:
json_path = "kannada_box_mapping.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(mapping_json, f, ensure_ascii=False, indent=4)

# Download JSON file
files.download(json_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from math import ceil, sqrt
import json
from google.colab import files

# -----------------------------
# Kannada classes metadata
# -----------------------------
CLASSES = {
    "swaras": [
        ("a", "ಅ", "0C85"), ("aa", "ಆ", "0C86"), ("i", "ಇ", "0C87"), ("ii", "ಈ", "0C88"),
        ("u", "ಉ", "0C89"), ("uu", "ಊ", "0C8A"), ("ru", "ಋ", "0C8B"), ("e", "ಎ", "0C8E"),
        ("ee", "ಏ", "0C8F"), ("ai", "ಐ", "0C90"), ("o", "ಒ", "0C92"), ("oo", "ಓ", "0C93"),
        ("au", "ಔ", "0C94"), ("am", "ಅಂ", "0C85+0C82"), ("aha", "ಅಃ", "0C85+0C83")
    ],
    "vyanjanas": [
        ("ka", "ಕ", "0C95"), ("kha", "ಖ", "0C96"), ("ga", "ಗ", "0C97"), ("gha", "ಘ", "0C98"), ("nga", "ಙ", "0C99"),
        ("cha", "ಚ", "0C9A"), ("chha", "ಛ", "0C9B"), ("ja", "ಜ", "0C9C"), ("jha", "ಝ", "0C9D"), ("nya", "ಞ", "0C9E"),
        ("ta_rt", "ಟ", "0C9F"), ("tha_rt", "ಠ", "0CA0"), ("da_rt", "ಡ", "0CA1"), ("dha_rt", "ಢ", "0CA2"), ("na_rt", "ಣ", "0CA3"),
        ("ta_dn", "ತ", "0CA4"), ("tha_dn", "ಥ", "0CA5"), ("da_dn", "ದ", "0CA6"), ("dha_dn", "ಧ", "0CA7"), ("na_dn", "ನ", "0CA8"),
        ("pa", "ಪ", "0CAA"), ("pha", "ಫ", "0CAB"), ("ba", "ಬ", "0CAC"), ("bha", "ಭ", "0CAD"), ("ma", "ಮ", "0CAE"),
        ("ya", "ಯ", "0CAF"), ("ra", "ರ", "0CB0"), ("la", "ಲ", "0CB2"), ("va", "ವ", "0CB5"),
        ("lla", "ಳ", "0CB3"), ("sha", "ಶ", "0CB6"), ("ssa", "ಷ", "0CB7"), ("sa", "ಸ", "0CB8"), ("ha", "ಹ", "0CB9")
    ],
    "matras": [
        ("aa_matra", "ಾ", ""), ("i_matra", "ಿ", ""), ("ii_matra", "ೀ", ""), ("u_matra", "ು", ""), ("uu_matra", "ೂ", ""),
        ("ru_matra", "ೃ", ""), ("e_matra", "ೆ", ""), ("ee_matra", "ೇ", ""), ("ai_matra", "ೈ", ""), ("o_matra", "ೊ", ""),
        ("oo_matra", "ೋ", ""), ("au_matra", "ೌ", ""), ("am_matra", "ಂ", ""), ("aha_matra", "ಃ", ""), ("virama", "್", "")
    ],
    "gunitaksharas": [
        ("kaa", "ಕಾ", ""), ("ki", "ಕಿ", ""), ("kii", "ಕೀ", ""), ("ku", "ಕು", ""), ("kuu", "ಕೂ", ""),
        ("kai", "ಕೈ", ""), ("ke", "ಕೆ", ""), ("ko", "ಕೊ", ""), ("kho", "ಖೊ", ""), ("gaai", "ಗೈ", ""),
        ("gi", "ಗಿ", ""), ("gu", "ಗು", ""), ("chi", "ಚಿ", ""), ("chu", "ಚು", ""), ("jaa", "ಜಾ", ""),
        ("ji", "ಜಿ", ""), ("ta_rt_i", "ಟಿ", ""), ("ta_dn_i", "ತಿ", ""), ("da_dn_uu", "ದು", ""), ("na_dn_ai", "ನೈ", ""),
        ("pa_i", "ಪಿ", ""), ("pa_oo", "ಪೋ", ""), ("ba_i", "ಬಿ", ""), ("ba_ee", "ಬೇ", ""), ("ma_oo", "ಮೋ", ""),
        ("ya_e", "ಯೆ", ""), ("ra_ai", "ರೈ", ""), ("la_o", "ಲೊ", ""), ("va_e", "ವೆ", ""), ("sa_i", "ಸಿ", ""),
        ("sha_i", "ಶಿ", ""), ("ha_ee", "ಹೇ", "")
    ],
    "vottaksharas": [
        ("kka", "ಕ್ಕ", ""), ("kta", "ಕ್ತ", ""), ("kna", "ಕ್ನ", ""), ("kra", "ಕ್ರ", ""), ("kma", "ಕ್ಮ", ""),
        ("gda", "ಗ್ದ", ""), ("gla", "ಗ್ಲ", ""), ("gra", "ಗ್ರ", ""), ("gga", "ಗ್ಗ", ""), ("gna", "ಗ್ನ", ""),
        ("cha_cha", "ಚ್ಚ", ""), ("chra", "ಛ್ರ", ""), ("jja", "ಜ್ಜ", ""), ("jra", "ಜ್ರ", ""), ("jna", "ಜ್ಞ", ""),
        ("tta", "ಟ್ಟ", ""), ("tna", "ಟ್ನ", ""), ("tra", "ತ್ರ", ""), ("tma", "ತ್ಮ", ""), ("tda", "ತ್ದ", ""),
        ("dda", "ದ್ದ", ""), ("dga", "ದ್ಗ", ""), ("dya", "ದ್ಯ", ""), ("dra", "ದ್ರ", ""), ("dha_na", "ಧಾನ", ""),
        ("ppa", "ಪ್ಪ", ""), ("pra", "ಪ್ರ", ""), ("pla", "ಪ್ಲ", ""), ("bba", "ಬ್ಬ", ""), ("bra", "ಬ್ರ", ""),
        ("bha_ra", "ಭ್ರ", ""), ("mra", "ಮ್ರ", ""), ("mma", "ಮ್ಮ", ""), ("mna", "ಮ್ನ", ""), ("ya_ya", "ಯ್ಯ", ""),
        ("yra", "ಯ್ರ", ""), ("ra_ra", "ರ್ರ", ""), ("rma", "ರ್ಮ", ""), ("rna", "ರ್ನ", ""), ("la_la", "ಲ್ಲ", ""),
        ("lra", "ಲ್ರ", ""), ("vra", "ವ್ರ", ""), ("ssa_ra", "ಷ್ರ", ""), ("sha_ra", "ಶ್ರ", ""), ("sna", "ಸ್ನ", ""),
        ("spa", "ಸ್ಪ", ""), ("ska", "ಸ್ಕ", ""), ("sma", "ಸ್ಮ", ""), ("hra", "ಹ್ರ", ""), ("hna", "ಹ್ನ", ""),
        ("hva", "ಹ್ವ", ""), ("hla", "ಹ್ಲ", ""), ("lla", "ಳ್ಳ", ""), ("nta", "ನ್ತ", ""), ("nda", "ನ್ದ", ""),
        ("nra", "ನ್ರ", ""), ("ksha", "ಕ್ಷ", ""), ("shcha", "ಶ್ಚ", ""), ("shna", "ಶ್ನ", ""), ("shma", "ಶ್ಮ", ""),
        ("dva", "ದ್ವ", ""), ("tva", "ತ್ವ", ""), ("bha_ra_alt", "ಭ್ರ", ""), ("sra", "ಸ್ರ", ""), ("kra_alt", "ಕ್ರ", "")
    ]
}

# -----------------------------
# Flatten for Sheet order
# -----------------------------
SHEET1_ORDER = (
    CLASSES["swaras"] +
    CLASSES["vyanjanas"] +
    CLASSES["matras"] +
    CLASSES["gunitaksharas"][:16]
)

SHEET2_ORDER = (
    CLASSES["gunitaksharas"][16:] +
    CLASSES["vottaksharas"]
)

# -----------------------------
# Generate mapping JSON
# -----------------------------
def generate_box_mapping(sheet_chars, sheet_name, cols=8):
    mapping = []
    rows = ceil(len(sheet_chars)/cols)
    box_width, box_height = 50, 50  # dummy placeholder values
    x_start, y_start = 0, 0  # for simplicity, coordinates can be adjusted later

    idx = 0
    for r in range(rows):
        for c in range(cols):
            if idx >= len(sheet_chars):
                break
            char_id, char_symbol, unicode_val = sheet_chars[idx]
            mapping.append({
                "name": char_id,
                "symbol": char_symbol,
                "category": sheet_name,
                "unicode": unicode_val,
                "coords": [x_start + c*box_width, y_start + r*box_height, box_width, box_height]
            })
            idx += 1
    return mapping

mapping_json = {
    "page1": generate_box_mapping(SHEET1_ORDER, "sheet1", cols=8),
    "page2": generate_box_mapping(SHEET2_ORDER, "sheet2", cols=8)
}

# -----------------------------
# Save & download JSON in Colab
# -----------------------------
json_path = "kannada_box_mapping_full.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(mapping_json, f, ensure_ascii=False, indent=4)

files.download(json_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from math import ceil, sqrt
import json
from google.colab import files

# -----------------------------
# Classes Metadata
# -----------------------------
CLASSES = {
    "swaras": [
        ("a", "ಅ", "0C85"), ("aa", "ಆ", "0C86"), ("i", "ಇ", "0C87"), ("ii", "ಈ", "0C88"),
        ("u", "ಉ", "0C89"), ("uu", "ಊ", "0C8A"), ("ru", "ಋ", "0C8B"), ("e", "ಎ", "0C8E"),
        ("ee", "ಏ", "0C8F"), ("ai", "ಐ", "0C90"), ("o", "ಒ", "0C92"), ("oo", "ಓ", "0C93"),
        ("au", "ಔ", "0C94"), ("am", "ಅಂ", "0C85+0C82"), ("aha", "ಅಃ", "0C85+0C83")
    ],
    "vyanjanas": [
        ("ka", "ಕ", "0C95"), ("kha", "ಖ", "0C96"), ("ga", "ಗ", "0C97"), ("gha", "ಘ", "0C98"), ("nga", "ಙ", "0C99"),
        ("cha", "ಚ", "0C9A"), ("chha", "ಛ", "0C9B"), ("ja", "ಜ", "0C9C"), ("jha", "ಝ", "0C9D"), ("nya", "ಞ", "0C9E"),
        ("ta_rt", "ಟ", "0C9F"), ("tha_rt", "ಠ", "0CA0"), ("da_rt", "ಡ", "0CA1"), ("dha_rt", "ಢ", "0CA2"), ("na_rt", "ಣ", "0CA3"),
        ("ta_dn", "ತ", "0CA4"), ("tha_dn", "ಥ", "0CA5"), ("da_dn", "ದ", "0CA6"), ("dha_dn", "ಧ", "0CA7"), ("na_dn", "ನ", "0CA8"),
        ("pa", "ಪ", "0CAA"), ("pha", "ಫ", "0CAB"), ("ba", "ಬ", "0CAC"), ("bha", "ಭ", "0CAD"), ("ma", "ಮ", "0CAE"),
        ("ya", "ಯ", "0CAF"), ("ra", "ರ", "0CB0"), ("la", "ಲ", "0CB2"), ("va", "ವ", "0CB5"),
        ("lla", "ಳ", "0CB3"), ("sha", "ಶ", "0CB6"), ("ssa", "ಷ", "0CB7"), ("sa", "ಸ", "0CB8"), ("ha", "ಹ", "0CB9")
    ],
    "matras": [
        ("aa_matra", "ಾ", ""), ("i_matra", "ಿ", ""), ("ii_matra", "ೀ", ""), ("u_matra", "ು", ""), ("uu_matra", "ೂ", ""),
        ("ru_matra", "ೃ", ""), ("e_matra", "ೆ", ""), ("ee_matra", "ೇ", ""), ("ai_matra", "ೈ", ""), ("o_matra", "ೊ", ""),
        ("oo_matra", "ೋ", ""), ("au_matra", "ೌ", ""), ("am_matra", "ಂ", ""), ("aha_matra", "ಃ", ""), ("virama", "್", "")
    ],
    "gunitaksharas": [
        ("kaa", "ಕಾ", ""), ("ki", "ಕಿ", ""), ("kii", "ಕೀ", ""), ("ku", "ಕು", ""), ("kuu", "ಕೂ", ""),
        ("kai", "ಕೈ", ""), ("ke", "ಕೆ", ""), ("ko", "ಕೊ", ""), ("kho", "ಖೊ", ""), ("gaai", "ಗೈ", ""),
        ("gi", "ಗಿ", ""), ("gu", "ಗು", ""), ("chi", "ಚಿ", ""), ("chu", "ಚು", ""), ("jaa", "ಜಾ", ""),
        ("ji", "ಜಿ", ""), ("ta_rt_i", "ಟಿ", ""), ("ta_dn_i", "ತಿ", ""), ("da_dn_uu", "ದು", ""), ("na_dn_ai", "ನೈ", ""),
        ("pa_i", "ಪಿ", ""), ("pa_oo", "ಪೋ", ""), ("ba_i", "ಬಿ", ""), ("ba_ee", "ಬೇ", ""), ("ma_oo", "ಮೋ", ""),
        ("ya_e", "ಯೆ", ""), ("ra_ai", "ರೈ", ""), ("la_o", "ಲೊ", ""), ("va_e", "ವೆ", ""), ("sa_i", "ಸಿ", ""),
        ("sha_i", "ಶಿ", ""), ("ha_ee", "ಹೇ", "")
    ],
    "vottaksharas": [
        ("kka", "ಕ್ಕ", ""), ("kta", "ಕ್ತ", ""), ("kna", "ಕ್ನ", ""), ("kra", "ಕ್ರ", ""), ("kma", "ಕ್ಮ", ""),
        ("gda", "ಗ್ದ", ""), ("gla", "ಗ್ಲ", ""), ("gra", "ಗ್ರ", ""), ("gga", "ಗ್ಗ", ""), ("gna", "ಗ್ನ", ""),
        ("cha_cha", "ಚ್ಚ", ""), ("chra", "ಛ್ರ", ""), ("jja", "ಜ್ಜ", ""), ("jra", "ಜ್ರ", ""), ("jna", "ಜ್ಞ", ""),
        ("tta", "ಟ್ಟ", ""), ("tna", "ಟ್ನ", ""), ("tra", "ತ್ರ", ""), ("tma", "ತ್ಮ", ""), ("tda", "ತ್ದ", ""),
        ("dda", "ದ್ದ", ""), ("dga", "ದ್ಗ", ""), ("dya", "ದ್ಯ", ""), ("dra", "ದ್ರ", ""), ("dha_na", "ಧಾನ", ""),
        ("ppa", "ಪ್ಪ", ""), ("pra", "ಪ್ರ", ""), ("pla", "ಪ್ಲ", ""), ("bba", "ಬ್ಬ", ""), ("bra", "ಬ್ರ", ""),
        ("bha_ra", "ಭ್ರ", ""), ("mra", "ಮ್ರ", ""), ("mma", "ಮ್ಮ", ""), ("mna", "ಮ್ನ", ""), ("ya_ya", "ಯ್ಯ", ""),
        ("yra", "ಯ್ರ", ""), ("ra_ra", "ರ್ರ", ""), ("rma", "ರ್ಮ", ""), ("rna", "ರ್ನ", ""), ("la_la", "ಲ್ಲ", ""),
        ("lra", "ಲ್ರ", ""), ("vra", "ವ್ರ", ""), ("ssa_ra", "ಷ್ರ", ""), ("sha_ra", "ಶ್ರ", ""), ("sna", "ಸ್ನ", ""),
        ("spa", "ಸ್ಪ", ""), ("ska", "ಸ್ಕ", ""), ("sma", "ಸ್ಮ", ""), ("hra", "ಹ್ರ", ""), ("hna", "ಹ್ನ", ""),
        ("hva", "ಹ್ವ", ""), ("hla", "ಹ್ಲ", ""), ("lla", "ಳ್ಳ", ""), ("nta", "ನ್ತ", ""), ("nda", "ನ್ದ", ""),
        ("nra", "ನ್ರ", ""), ("ksha", "ಕ್ಷ", ""), ("shcha", "ಶ್ಚ", ""), ("shna", "ಶ್ನ", ""), ("shma", "ಶ್ಮ", ""),
        ("dva", "ದ್ವ", ""), ("tva", "ತ್ವ", ""), ("bha_ra_alt", "ಭ್ರ", ""), ("sra", "ಸ್ರ", ""), ("kra_alt", "ಕ್ರ", "")
    ]
}

# -----------------------------
# Flatten for 2 sheets
# -----------------------------
SHEET1_ORDER = CLASSES["swaras"] + CLASSES["vyanjanas"] + CLASSES["matras"] + CLASSES["gunitaksharas"][:16]
SHEET2_ORDER = CLASSES["gunitaksharas"][16:] + CLASSES["vottaksharas"]

# -----------------------------
# Draw PDF & generate mapping
# -----------------------------
def draw_page(c, chars, page_title, cols=8):
    width, height = A4
    margin = 10*mm
    spacing = 4*mm
    n_boxes = len(chars)
    rows = ceil(n_boxes / cols)

    # compute box size
    box_width = (width - 2*margin - (cols-1)*spacing) / cols
    box_height = (height - 2*margin - 20*mm - (rows-1)*spacing) / rows
    box_size = min(box_width, box_height)

    x_start = (width - (cols*box_size + (cols-1)*spacing)) / 2
    y_start = height - margin - 15*mm

    c.setFont("Helvetica-Bold", 16)
    c.drawCentredString(width/2, height-margin, page_title)

    mapping = []
    idx = 0
    for r in range(rows):
        y = y_start - r*(box_size + spacing)
        for col in range(cols):
            if idx >= n_boxes:
                break
            x = x_start + col*(box_size + spacing)
            c.rect(x, y - box_size, box_size, box_size)

            char_id, char_symbol, unicode_val = chars[idx]
            mapping.append({
                "name": char_id,
                "symbol": char_symbol,
                "category": page_title,
                "unicode": unicode_val,
                "coords": [x, y - box_size, box_size, box_size]
            })
            idx += 1
    return mapping

# -----------------------------
# Create PDF & mapping
# -----------------------------
OUTPUT_PDF = "kannada_161_boxes.pdf"
c = canvas.Canvas(OUTPUT_PDF, pagesize=A4)

mapping1 = draw_page(c, SHEET1_ORDER, "Sheet 1", cols=8)
c.showPage()
mapping2 = draw_page(c, SHEET2_ORDER, "Sheet 2", cols=8)
c.showPage()
c.save()

# Save mapping JSON
mapping_json = {"page1": mapping1, "page2": mapping2}
json_path = "kannada_box_mapping_161.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(mapping_json, f, ensure_ascii=False, indent=4)

# Download both PDF & JSON
files.download(OUTPUT_PDF)
files.download(json_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pdf2image pillow numpy tqdm gdown
!sudo apt-get install poppler-utils  # for pdf2image backend


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (450 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126675

In [ ]:
import os, re, math, shutil
from pdf2image import convert_from_path
from PIL import Image, ImageOps
import numpy as np
from tqdm import tqdm
import gdown

# ---------- CONFIG ----------
DRIVE_FOLDER_URL = "https://drive.google.com/drive/folders/1eJDi3sUmfQkWqJiDGGGITzNfZJaufXs8?usp=drive_link"  # 🔁 paste your Google Drive folder link here
DOWNLOAD_DIR = "downloaded_pdfs"
OUTPUT_DIR   = "Kannada_Dataset"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------- STEP 1: Download PDFs from Google Drive ----------
def download_from_drive_folder(folder_url, output_dir):
    """
    Downloads all files from a public Google Drive folder using gdown.
    """
    print(f"📥 Downloading all PDFs from: {folder_url}")
    gdown.download_folder(url=folder_url, output=output_dir, quiet=False, use_cookies=False)
    print(f"✅ All PDFs downloaded into: {output_dir}")

# ---------- STEP 2: A4 Grid Geometry ----------
A4_W, A4_H = 595, 842
mm2pt = 2.83465
rows, cols = 5, 10
box_w, box_h = 16*mm2pt, 30*mm2pt
space_x, space_y = 2*mm2pt, 4*mm2pt
total_grid_w = cols*box_w + (cols-1)*space_x
start_x = (A4_W - total_grid_w)/2
start_y = A4_H - 50*mm2pt - 10
pad_x_pt, pad_y_pt = 1.5*mm2pt, 1.5*mm2pt

# ---------- STEP 3: Image Utilities ----------
def clean_crop(img_pil):
    """Trim borders and enhance contrast."""
    gray = img_pil.convert("L")
    arr = np.array(gray)
    thresh = np.mean(arr) - 30
    mask = arr < thresh
    if mask.any():
        ys, xs = np.where(mask)
        x0, x1 = xs.min(), xs.max()
        y0, y1 = ys.min(), ys.max()
        cropped = img_pil.crop((x0, y0, x1, y1))
    else:
        cropped = img_pil
    return ImageOps.autocontrast(cropped)

def parse_filename(fname):
    """
    Example: VOTTAKSHARAS_139_ssa_ra.pdf
    → category='VOTTAKSHARAS', char_id='139_ssa_ra'
    """
    base = os.path.splitext(os.path.basename(fname))[0]
    parts = base.split("_", 2)
    if len(parts) >= 3:
        category = parts[0]
        char_id  = parts[1] + "_" + parts[2]
    else:
        category = parts[0]
        char_id  = base
    return category, char_id

# ---------- STEP 4: Core Cropper ----------
def process_pdf(pdf_path):
    category, char_id = parse_filename(pdf_path)
    out_dir = os.path.join(OUTPUT_DIR, category, char_id)
    os.makedirs(out_dir, exist_ok=True)

    pages = convert_from_path(pdf_path, dpi=300)
    if not pages:
        print(f"⚠️ No pages found in {pdf_path}")
        return

    img = pages[0].convert("RGB")
    W_px, H_px = img.size
    sx, sy = W_px / A4_W, H_px / A4_H

    idx = 1
    for i in range(rows):
        for j in range(cols):
            x_pt = start_x + j * (box_w + space_x) + pad_x_pt
            y_pt_top = start_y - i * (box_h + space_y) - pad_y_pt
            x2_pt = x_pt + box_w - 2*pad_x_pt
            y2_pt = y_pt_top - (box_h - 2*pad_y_pt)

            left   = int(x_pt * sx)
            right  = int(x2_pt * sx)
            top    = int((A4_H - y_pt_top) * sy)
            bottom = int((A4_H - y2_pt) * sy)

            crop = img.crop((left, top, right, bottom))
            crop = clean_crop(crop)

            out_path = os.path.join(out_dir, f"{char_id}_{idx:02d}.png")
            crop.save(out_path)
            idx += 1
    return idx - 1

# ---------- STEP 5: Master Runner ----------
def build_dataset():
    # Step 1: Download
    download_from_drive_folder(DRIVE_FOLDER_URL, DOWNLOAD_DIR)

    # Step 2: Extract
    pdf_files = sorted([f for f in os.listdir(DOWNLOAD_DIR) if f.lower().endswith(".pdf")])
    for pdf_file in tqdm(pdf_files, desc="🖼️ Extracting sheets"):
        full_path = os.path.join(DOWNLOAD_DIR, pdf_file)
        try:
            n = process_pdf(full_path)
            print(f"✅ {pdf_file} → {n} crops saved")
        except Exception as e:
            print(f"❌ Error processing {pdf_file}: {e}")

    print(f"\n🎉 Dataset built successfully at: {OUTPUT_DIR}")

# ---------- RUN ----------
if __name__ == "__main__":
    build_dataset()


📥 Downloading all PDFs from: https://drive.google.com/drive/folders/1eJDi3sUmfQkWqJiDGGGITzNfZJaufXs8?usp=drive_link


Retrieving folder contents


Retrieving folder 1hU7vxw72C6F-WCmQJmMbakSRoj8-jfVe Gunitaksharas 
Processing file 1x-uBSYVxyXaUlOcAljiFTrfOYvcH84ps GUNITAKSHARA_050_maa.pdf
Processing file 1Unvh-Mrba0LDi4WGfUOBEp_E7YMSqT3Q GUNITAKSHARAS _075_gam.pdf
Processing file 1PvGJ_GqR-il05Sb6P45WkeBTh1r5NVAY GUNITAKSHARAS_061_bhru.pdf
Processing file 1oyxeA2AZdOD4ELgWZNDMoIfmobBgfW6G GUNITAKSHARAS_bo2_071.pdf
Processing file 1Gw1rU3A6rWgIIasVRj8jNwiOdmXrEwYL GUNITAKSHARAS_che_063.pdf
Processing file 19enGtLIk2ZKBJEdly_aloKRZz57njLca GUNITAKSHARAS_ddo2_072.pdf
Processing file 1BeQHIX5yE9nww4cc-BKzrGCMGM5PS9IQ GUNITAKSHARAS_de2_066.pdf
Processing file 1rbe8G_HDk9lzgl5xlU0YI_DNq-FsIwUg GUNITAKSHARAS_gho_070.pdf
Processing file 1dQRhzJhKbq_Up3v-4o8S4ziF4ILrHn-v GUNITAKSHARAS_gi_052.pdf
Processing file 1ktjGFU5fTXd9qyExD2yKTRQ-o9xsJ5Fp GUNITAKSHARAS_goo_058.pdf
Processing file 1BYY9iGhbXj2SOXwiJAtNBtM9yYIhbOUC GUNITAKSHARAS_haha_077.pdf
Processing file 1cx9rVPrCvCwYNJxUFrvyLG6bzB3jLen_ GUNITAKSHARAS_jai_068.pdf
Processing file 1_9

FolderContentsMaximumLimitError: The gdrive folder with url: https://drive.google.com/drive/folders/1BJs-RkjqQgHjuuLRYopCIBxtoY7DaYLs?hl=en has more than 50 files, gdrive can't download more than this limit.